In [3]:
import pandas as pd

In [4]:

raw_texto_categoria_productos = pd.read_csv('../datasets/raw/ProductCategoryText.csv')
raw_productos = pd.read_csv('../datasets/raw/Products.csv')
raw_texto_productos = pd.read_csv('../datasets/raw/ProductTexts.csv')
raw_orden_ventas_item = pd.read_csv('../datasets/raw/SalesOrderItems.csv')

In [5]:
def eda(df, nombre, id='id'):
    print(f'EDA de {nombre} \n')
    print('Primeras filas:')
    print(df.head())
    print()
    
    print(df.info())
    print()

    print('Filas enteras repeditas:', df.duplicated().sum())
    print()

    print('Filas con id repetido:', df.duplicated(id).sum())
    print()

    print('Valores ausentes:')
    print(df.isna().sum())
    print()

## Ordenes de Venta por Item

In [6]:
eda(raw_orden_ventas_item, "raw_orden_ventas_item", "PRODUCTID")

EDA de raw_orden_ventas_item 

Primeras filas:
   SALESORDERID  SALESORDERITEM PRODUCTID NOTEID CURRENCY  GROSSAMOUNT  \
0     500000000              10   MB-1034             USD         2499   
1     500000000              20   CB-1161             USD          399   
2     500000001              10   HB-1175             USD          899   
3     500000001              20   RC-1056             USD         2499   
4     500000001              30   CC-1021             USD         1144   

   NETAMOUNT  TAXAMOUNT ITEMATPSTATUS  OPITEMPOS  QUANTITY QUANTITYUNIT  \
0   2186.625    312.375             I        NaN         4           EA   
1    349.125     49.875             I        NaN         9           EA   
2    786.625    112.375             I        NaN         2           EA   
3   2186.625    312.375             I        NaN         2           EA   
4   1001.000    143.000             I        NaN         3           EA   

   DELIVERYDATE  
0      20180311  
1      20180311  
2  

1. Conversión de fechas

In [7]:
raw_orden_ventas_item['DELIVERYDATE'] = pd.to_datetime(raw_orden_ventas_item['DELIVERYDATE'], format='%Y%m%d', errors='coerce')


Se convierte la columna DELIVERYDATE a formato de fecha (datetime) para poder realizar análisis temporales, agrupar por semana, mes o año, y preparar variables de tiempo para el modelado.

2. Eliminación de columnas innecesarias

In [8]:
columnas_a_eliminar = [
    'QUANTITYUNIT',
    'NOTEID',
    'TAXAMOUNT',
    'ITEMATPSTATUS',
    'SALESORDERID',
    'SALESORDERITEM',
    'CURRENCY',
    'NETAMOUNT',
    'OPITEMPOS'
]

raw_orden_ventas_item.drop(columns=[col for col in columnas_a_eliminar if col in raw_orden_ventas_item.columns], inplace=True)



Se eliminan columnas que no aportan valor al análisis ni al modelo, como identificadores, códigos de posición, información duplicada o logística, y columnas con valores constantes o sin uso. Esto simplifica el dataset y mejora la interpretabilidad.

3. Tratamiento de valores nulos

In [9]:
raw_orden_ventas_item.dropna(subset=['QUANTITY', 'DELIVERYDATE', 'PRODUCTID', 'GROSSAMOUNT'], inplace=True)


Se eliminan las filas que tienen valores nulos en columnas clave como PRODUCTID, QUANTITY, DELIVERYDATE o GROSSAMOUNT, ya que estos campos son necesarios para calcular ingresos, identificar productos y relacionar con otras tablas.

4. Transformación de tipos de datos

In [10]:

raw_orden_ventas_item['GROSSAMOUNT'] = raw_orden_ventas_item['GROSSAMOUNT'].astype(int)
raw_orden_ventas_item['QUANTITY'] = raw_orden_ventas_item['QUANTITY'].astype(int)


Se transforma QUANTITY, GROSSAMOUNT a tipo entero, ya que representa unidades enteras vendidas.

5. Detección y eliminación de outliers

In [11]:
Q1 = raw_orden_ventas_item['QUANTITY'].quantile(0.25)
Q3 = raw_orden_ventas_item['QUANTITY'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

raw_orden_ventas_item = raw_orden_ventas_item[
    (raw_orden_ventas_item['QUANTITY'] >= limite_inferior) &
    (raw_orden_ventas_item['QUANTITY'] <= limite_superior)
]


Se identifican y eliminan valores atípicos en QUANTITY utilizando el rango intercuartílico (IQR), con el objetivo de evitar que cantidades extremadamente altas o bajas distorsionen el análisis o el entrenamiento del modelo.

6. Renombramiento de columna

In [12]:
raw_orden_ventas_item.rename(columns={'GROSSAMOUNT': 'PRICE_UNITARIO'}, inplace=True)

Se renombra la columna GROSSAMOUNT como PRICE_UNITARIO para reflejar su verdadero significado: el precio unitario del producto vendido. Esto mejora la claridad del análisis y evita confusión con montos totales.

In [13]:
print(raw_orden_ventas_item)

     PRODUCTID  PRICE_UNITARIO  QUANTITY DELIVERYDATE
0      MB-1034            2499         4   2018-03-11
1      CB-1161             399         9   2018-03-11
2      HB-1175             899         2   2018-02-28
3      RC-1056            2499         2   2018-02-28
4      CC-1021            1144         3   2018-02-28
...        ...             ...       ...          ...
1925   MB-1031             649        10   2019-07-09
1926   BX-1013             449         3   2019-07-09
1927   EB-1132            1900         3   2019-07-09
1928   BX-1011             249         5   2019-07-09
1929   BX-1015             299         6   2019-07-09

[1908 rows x 4 columns]


In [35]:
raw_orden_ventas_item.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1908 entries, 0 to 1929
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PRODUCTID       1908 non-null   object        
 1   PRICE_UNITARIO  1908 non-null   int32         
 2   QUANTITY        1908 non-null   int32         
 3   DELIVERYDATE    1908 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int32(2), object(1)
memory usage: 59.6+ KB


## Text Category Products


In [14]:
eda(raw_texto_categoria_productos, "raw_texto_categoria_productos", "PRODCATEGORYID")

EDA de raw_texto_categoria_productos 

Primeras filas:
  PRODCATEGORYID LANGUAGE       SHORT_DESCR  MEDIUM_DESCR  LONG_DESCR
0             RO       EN         Road Bike           NaN         NaN
1             BX       EN               BMX           NaN         NaN
2             CC       EN  Cyclo-cross Bike           NaN         NaN
3             MB       EN     Mountain Bike           NaN         NaN
4             RC       EN       Racing Bike           NaN         NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PRODCATEGORYID  9 non-null      object 
 1   LANGUAGE        9 non-null      object 
 2   SHORT_DESCR     9 non-null      object 
 3   MEDIUM_DESCR    0 non-null      float64
 4   LONG_DESCR      0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 492.0+ bytes
None

Filas enteras repeditas: 0

Filas con id repet

1. Eliminación de columnas innecesarias

In [15]:
columnas_a_eliminar = ['LANGUAGE', 'MEDIUM_DESCR', 'LONG_DESCR']
raw_texto_categoria_productos.drop(columns=columnas_a_eliminar, inplace=True)

Se eliminan las columnas LANGUAGE, MEDIUM_DESCR y LONG_DESCR ya que no contienen información útil. Dos de ellas están completamente vacías y el idioma no será utilizado para segmentación.

2. Eliminación de valores nulos

In [16]:

raw_texto_categoria_productos.dropna(subset=['SHORT_DESCR'], inplace=True)


Se eliminan registros que no contienen SHORT_DESCR, ya que esta columna representa la descripción principal que se usará en visualizaciones y reportes para identificar la categoría del producto.

3. Eliminación de duplicados

In [17]:

raw_texto_categoria_productos.drop_duplicates(subset='PRODCATEGORYID', keep='first', inplace=True)


Aunque no se detectaron duplicados en el EDA, se asegura que cada categoría tenga una única descripción asociada por PRODCATEGORYID, evitando errores en la unión con otras tablas.

4. Renombramiento de columna descriptiva

In [18]:

raw_texto_categoria_productos.rename(columns={'SHORT_DESCR': 'SHORT_DESCR_CAT'}, inplace=True)


Se renombra SHORT_DESCR a SHORT_DESCR_CAT para evitar confusión con la descripción de productos. Esto permite distinguir claramente entre descripciones de productos (SHORT_DESCR) y de categorías (SHORT_DESCR_CAT) al unir tablas o generar reportes.

In [19]:
print(raw_texto_categoria_productos)

  PRODCATEGORYID   SHORT_DESCR_CAT
0             RO         Road Bike
1             BX               BMX
2             CC  Cyclo-cross Bike
3             MB     Mountain Bike
4             RC       Racing Bike
5             DB     Downhill Bike
6             EB             eBike
7             CB           Cruiser
8             HB       Hybrid Bike


## Products


In [20]:
eda(raw_productos, "raw_productos", "PRODUCTID")

EDA de raw_productos 

Primeras filas:
  PRODUCTID TYPECODE PRODCATEGORYID  CREATEDBY  CREATEDAT  CHANGEDBY  \
0   RO-1001       PR             RO          9   20181003          9   
1   RO-1002       PR             RO          9   20181003          9   
2   RO-1003       PR             RO         12   20181003         12   
3   BX-1011       PR             BX          9   20181003          9   
4   BX-1012       PR             BX          6   20181003          6   

   CHANGEDAT  SUPPLIER_PARTNERID  TAXTARIFFCODE QUANTITYUNIT  WEIGHTMEASURE  \
0   20181003           100000000              1           EA            7.7   
1   20181003           100000001              1           EA            8.0   
2   20181003           100000002              1           EA            9.1   
3   20181003           100000003              1           EA           11.1   
4   20181003           100000004              1           EA           12.0   

  WEIGHTUNIT CURRENCY  PRICE  WIDTH  DEPTH  HEIGHT  D

1. Eliminación de columnas innecesarias

In [21]:
columnas_a_eliminar = [
    'PRODUCTPICURL',
    'WIDTH',
    'DEPTH',
    'HEIGHT',
    'DIMENSIONUNIT',
    'CHANGEDAT',
    'SUPPLIER_PARTNERID',
    'TAXTARIFFCODE',
    'QUANTITYUNIT',
    'WEIGHTMEASURE',
    'CHANGEDBY',
    'CREATEDBY',
    'TYPECODE',
    'CREATEDAT',
    'WEIGHTUNIT',
    'CURRENCY'
]
raw_productos.drop(columns=[col for col in columnas_a_eliminar if col in raw_productos.columns], inplace=True)

Se eliminan columnas técnicas o vacías como dimensiones físicas, identificadores internos, y campos de auditoría que no aportan al modelo de predicción de ventas. Esto reduce ruido y simplifica el dataset.

2. Eliminación de valores nulos

In [22]:
raw_productos.dropna(subset=['PRODUCTID', 'PRODCATEGORYID', 'PRICE'], inplace=True)

Se eliminan filas que no tienen información esencial en las columnas PRODUCTID, PRODCATEGORYID o PRICE, ya que son claves para enlazar con otras tablas y para calcular métricas de valor e inventario.

3. Conversión de tipo en la columna PRICE

In [23]:
raw_productos['PRICE'] = raw_productos['PRICE'].astype(int)

Se convierte la columna PRICE a tipo entero ya que representa precios definidos sin decimales. Esto estandariza el tipo de dato y evita errores al momento de agrupar, calcular totales o unir con otras tablas.

In [24]:
print(raw_productos)

   PRODUCTID PRODCATEGORYID  PRICE
0    RO-1001             RO    525
1    RO-1002             RO    689
2    RO-1003             RO    721
3    BX-1011             BX    249
4    BX-1012             BX    399
5    BX-1013             BX    449
6    BX-1014             BX    799
7    BX-1015             BX    299
8    BX-1016             BX    319
9    CC-1021             CC   1144
10   CC-1022             CC   1200
11   CC-1023             CC   1361
12   MB-1031             MB    649
13   MB-1032             MB   1299
14   MB-1033             MB   3999
15   MB-1034             MB   2499
16   RC-1051             RC   2499
17   RC-1052             RC   3999
18   RC-1053             RC   4599
19   RC-1054             RC   5499
20   RC-1055             RC   1999
21   RC-1056             RC   2499
22   RC-1057             RC   4999
23   DB-1081             DB   1499
24   DB-1082             DB   1250
25   DB-1083             DB   1199
26   EB-1131             EB   1500
27   EB-1132        

## Texto Productos

In [25]:
eda(raw_texto_productos, "raw_texto_productos", "PRODUCTID")

EDA de raw_texto_productos 

Primeras filas:
  PRODUCTID LANGUAGE       SHORT_DESCR MEDIUM_DESCR  LONG_DESCR
0   RO-1001       EN        Roady 1001          NaN         NaN
1   RO-1002       EN        Roady 1002          NaN         NaN
2   RO-1003       EN        Roady 1003          NaN         NaN
3   BX-1011       EN  BMX Vintage 1011          NaN         NaN
4   BX-1012       EN     BMX Jump 1012          NaN         NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PRODUCTID     44 non-null     object 
 1   LANGUAGE      44 non-null     object 
 2   SHORT_DESCR   44 non-null     object 
 3   MEDIUM_DESCR  36 non-null     object 
 4   LONG_DESCR    0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 1.8+ KB
None

Filas enteras repeditas: 0

Filas con id repetido: 2

Valores ausentes:
PRODUCTID        0
LANGUAGE         0

1. Eliminación de columnas innecesarias

In [26]:
columnas_a_eliminar = ['LANGUAGE', 'MEDIUM_DESCR', 'LONG_DESCR']
raw_texto_productos.drop(columns=columnas_a_eliminar, inplace=True)


Se eliminan las columnas LANGUAGE, MEDIUM_DESCR y LONG_DESCR ya que no aportan valor al análisis. El idioma no es relevante para este proyecto y las otras columnas contienen muchos valores nulos o son redundantes frente a SHORT_DESCR.

2. Eliminación de duplicados

In [27]:

raw_texto_productos.drop_duplicates(subset='PRODUCTID', keep='first', inplace=True)

Se conservan solo las primeras apariciones por PRODUCTID, ya que la relación entre un producto y su descripción debe ser única para evitar errores al unir con otras tablas.

3. Eliminación de valores nulos

In [28]:
raw_texto_productos.dropna(subset=['SHORT_DESCR'], inplace=True)

Se eliminan filas que no contienen una descripción corta (SHORT_DESCR), ya que esta será la columna principal para representar el nombre del producto en visualizaciones o reportes.

In [29]:
print(raw_texto_productos)

   PRODUCTID         SHORT_DESCR
0    RO-1001          Roady 1001
1    RO-1002          Roady 1002
2    RO-1003          Roady 1003
3    BX-1011    BMX Vintage 1011
4    BX-1012       BMX Jump 1012
5    BX-1013      BMX Jump Lux I
6    BX-1014     BMX Jump Lux II
7    BX-1015          BMX Optima
8    BX-1016       BMX Optima II
9    CC-1021       Cyclone Basic
10   CC-1022       Cyclone Speed
11   CC-1023         Cyclone III
12   MB-1031      Mt Discovery B
13   MB-1032  Mt Discovery Drive
14   MB-1033   Mt Discovery Rush
15   MB-1034   Mt Discovery Ulti
16   RC-1051           Tornado I
17   RC-1052          Tornado II
18   RC-1053            Stream I
19   RC-1054           Stream II
20   RC-1055           Veloflash
22   RC-1056       Veloflash SE 
24   RC-1057  Veloflash Ultimate
25   DB-1081         Rooty Basic
26   DB-1082           Capricorn
27   DB-1083        Capricorn II
28   EB-1131         Flash Drive
29   EB-1132      Flash Drive II
30   EB-1133            Lazy Cat
31   EB-11

## Integración de tablas

1. Crear la tabla de hechos fct_ventas

In [30]:

fct_ventas = raw_orden_ventas_item.merge(raw_productos, on='PRODUCTID', how='left')


Se crea la tabla fct_ventas al unir los detalles de las órdenes (raw_orden_ventas_item) con los productos (raw_productos). Esta tabla representa los hechos observables: cantidad vendida, producto, precio unitario y fecha de entrega. Será la base para calcular ventas, inventario y otras métricas clave.

In [31]:
print(fct_ventas)

     PRODUCTID  PRICE_UNITARIO  QUANTITY DELIVERYDATE PRODCATEGORYID  PRICE
0      MB-1034            2499         4   2018-03-11             MB   2499
1      CB-1161             399         9   2018-03-11             CB    399
2      HB-1175             899         2   2018-02-28             HB    899
3      RC-1056            2499         2   2018-02-28             RC   2499
4      CC-1021            1144         3   2018-02-28             CC   1144
...        ...             ...       ...          ...            ...    ...
1903   MB-1031             649        10   2019-07-09             MB    649
1904   BX-1013             449         3   2019-07-09             BX    449
1905   EB-1132            1900         3   2019-07-09             EB   1900
1906   BX-1011             249         5   2019-07-09             BX    249
1907   BX-1015             299         6   2019-07-09             BX    299

[1908 rows x 6 columns]


In [36]:
fct_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1908 entries, 0 to 1907
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PRODUCTID       1908 non-null   object        
 1   PRICE_UNITARIO  1908 non-null   int32         
 2   QUANTITY        1908 non-null   int32         
 3   DELIVERYDATE    1908 non-null   datetime64[ns]
 4   PRODCATEGORYID  1908 non-null   object        
 5   PRICE           1908 non-null   int32         
dtypes: datetime64[ns](1), int32(3), object(2)
memory usage: 67.2+ KB


2. Crear la tabla dimensional dim_productos

In [32]:
dim_productos = raw_productos.merge(
    raw_texto_productos[['PRODUCTID', 'SHORT_DESCR']], on='PRODUCTID', how='left'
)

dim_productos = dim_productos.merge(
    raw_texto_categoria_productos[['PRODCATEGORYID', 'SHORT_DESCR_CAT']], on='PRODCATEGORYID', how='left'
)


Se construye la tabla dim_productos combinando la información técnica del producto (raw_productos) con su descripción (raw_texto_productos) y la descripción de su categoría (raw_texto_categoria_productos). Esta dimensión permite interpretar los códigos y clasificar los productos de manera comprensible en reportes o visualizaciones.

In [33]:
print(dim_productos)

   PRODUCTID PRODCATEGORYID  PRICE         SHORT_DESCR   SHORT_DESCR_CAT
0    RO-1001             RO    525          Roady 1001         Road Bike
1    RO-1002             RO    689          Roady 1002         Road Bike
2    RO-1003             RO    721          Roady 1003         Road Bike
3    BX-1011             BX    249    BMX Vintage 1011               BMX
4    BX-1012             BX    399       BMX Jump 1012               BMX
5    BX-1013             BX    449      BMX Jump Lux I               BMX
6    BX-1014             BX    799     BMX Jump Lux II               BMX
7    BX-1015             BX    299          BMX Optima               BMX
8    BX-1016             BX    319       BMX Optima II               BMX
9    CC-1021             CC   1144       Cyclone Basic  Cyclo-cross Bike
10   CC-1022             CC   1200       Cyclone Speed  Cyclo-cross Bike
11   CC-1023             CC   1361         Cyclone III  Cyclo-cross Bike
12   MB-1031             MB    649      Mt Discover

## Exportación de tablas limpias

In [37]:
# Renombrar las tablas limpias
clean_orden_ventas_item = raw_orden_ventas_item
clean_productos = raw_productos
clean_texto_productos = raw_texto_productos
clean_texto_categoria_productos = raw_texto_categoria_productos

# Guardar archivos CSV 
clean_orden_ventas_item.to_csv('../datasets/clean/clean_orden_ventas_item.csv', index=False)
clean_productos.to_csv('../datasets/clean/clean_productos.csv', index=False)
clean_texto_productos.to_csv('../datasets/clean/clean_texto_productos.csv', index=False)
clean_texto_categoria_productos.to_csv('../datasets/clean/clean_texto_categoria_productos.csv', index=False)

# Guardar archivos CSV de las tablas finales
fct_ventas.to_csv('../datasets/clean/fct_ventas.csv', index=False)
dim_productos.to_csv('../datasets/clean/dim_productos.csv', index=False)


## Resumen final del proceso de Data Wrangling

Durante esta etapa se llevó a cabo la limpieza, transformación y organización de los datos crudos provenientes del sistema SAP, con el objetivo de preparar la base para el modelado de predicción de ventas e inventario. El proceso incluyó:

Selección y depuración de tablas relevantes, descartando aquellas irrelevantes o redundantes.

Limpieza estructurada de cada tabla, eliminando columnas sin utilidad, valores nulos y duplicados.

Estandarización de formatos de fecha, tipos de datos y nombres de columnas para garantizar consistencia.

Construcción de dos tablas clave:

fct_ventas: tabla de hechos con el detalle por producto, cantidad y precio.

dim_productos: tabla dimensional con descripciones legibles de productos y categorías.

Finalmente, todas las tablas limpias y transformadas fueron exportadas como archivos CSV a la carpeta ../datasets/clean/, y están listas para ser utilizadas en la siguiente etapa del proyecto: Feature Engineering y Modelado.